# Recognizing dogs and cats

The purpose of this laboratory is to build a first end to end reflex-based AI model to teach computers to [**understand images**](https://www.ted.com/talks/fei_fei_li_how_we_re_teaching_computers_to_understand_pictures).

In particular, the objective of this lab is to write an AI application able to recognize cats and dogs on images. Your application will take an image as input and will be able to say wheter the image contains a dog or a cat. You will work with the data of the [**Dogs vs Cats**](https://www.kaggle.com/c/dogs-vs-cats) competition from Kaggle. This competition was launched in 2013 and the first place was obtained by [Pierre Sermanet](https://research.google.com/pubs/PierreSermanet.html), actually Research Scientist at Google Brain, by using the [Overfeat](http://cilvr.nyu.edu/doku.php?id=software:overfeat:start#overfeatobject_recognizer_feature_extractor) deep learning library he wrote during his PhD at New York University under the supervision of [Yann Le Cun](http://yann.lecun.com/), Director of AI Research at Facebook. He obtained $1.09%$ of classification errors. Try to do your best to approach this score!!!

Two approaches will be used to adress this problem :
1. A traditional pattern recognition model in which hand-crafted features are extracted from images and used to represent them and to train classifiers.
2. A modern representation learning approach in which deep convolutional neural networks (CNN) are used to learn the image representations.

 

##  Learning outcomes
+ Building an end to end supervised machine learning pipeline : input data (training set) preparation, training (model learning), validation sets, cross-validation, hyper-parameter tuning, evaluation on the testing dataset.
+ Getting familiar with deep learning for image classification : model building and training, transfer learning, fine-tuning.
+ Getting familiar with some well-known librairies:
    + Machine learning : Scikit-learn ([http://scikit-learn.org/stable/](http://scikit-learn.org/stable/))
    + Deep learning: Keras ([https://keras.io/](https://keras.io/))
    + Computer vision : Scikit-image ([http://scikit-image.org/](http://scikit-image.org/) or OpenCV ([http://opencv.org/](http://opencv.org/))
    
**The final objective of this laboratory is to be aware to the potential but also to the limitations of reflex-based AI approaches towards visual recognition tasks.**
    

## Part 0 : Requirements
A set of packages will be useful to handle the first part of this study case.

    pip install -r requirements.txt

In [ ]:
!pip install -r requirements.txt

## Part 1 : A small tutorial on image classification

In this section, we will briefly introduce the image classification problem which consists in assigning to an input image one label from a fixed set of labels and which is one of the big challenge of computer vision and artifial intelligence. In our case, we will only consider two labels $\{dog, cat\}$. This small tutorial also aims at familiarizing you with machine learning and computer vision librairies that we will used in this course :
+ Scikit-Learn : [http://scikit-learn.org/stable/](http://scikit-learn.org/stable/)
+ OpenCV : [http://opencv.org/](http://opencv.org/) or Scikit-image ([http://scikit-image.org/](http://scikit-image.org/)).

![ImageCat](images/Diapositive1.jpg)

While the task of image classification is very easy for a human, we have to face with several challenge to build our automatic recognition algorithm among whom:

+ Viewpoint variation.
+ Scale variation.
+ Illumination conditions variation.
+ Deformation.
+ Occlusion.
+ Backgroud clutter.
+ Intraclass variation.

![ImageCatwithvariations](./images/Diapositive2.jpg)

![ImageCatwithocclusion](./images/Diapositive3.jpg)

Source : Images from the CS231n course of Stanford (Convolutional Neural Networks for Visual Recognition)












### A simple image classification pipeline

To built our image classification algorithm, we will follow the principle of a machine learning approach for image classification which consists in :
1. Collecting and preparing a dataset of images and their corresponding labels.
2. Using a machine learning algorithm to train a classifier.
3. Evaluate the classifier on new images.


![ImageClassificationpipeline](images/testphase.png)

#### Having a look on the available data

First, you have to  download the dataset that will be used to train and test our model. Before downloading the data, create a subdirectory in your working folder called data. Then download [DataDogsCatsChallenge.zip](https://filesender.renater.fr/?s=download&token=cd3d55af-2563-47c4-af3c-28a4eba661e7) into that directory and unzip it. This dataset contains 25,000 labelled dog and cat photos available for training, and 12,500 in the test set that we have to try to label for the Kaggle competition.

As we have seen in the lecture note, the standard practice in machine learning is to split the available data into at least two different subsets :
+ The **training set** : to learn the model.
+ The **testing set** : to evaluate the learned model.

You have also seen that is the also standard to add a third set to the split to build a **validation set** that will be used to fine-tune the parameters of the model.

I you have a look on the DogsCatsChallenge directory, you will see that the preparation of the data have been done and that the test and train sets are in separate subdirectories in which data for each category (cats and dogs) is also into subdirectories. Nevertheless, there is no validation set and one of your first task will be to build it.

The archive also contains a directory named **sample**. Training and validating the entire dataset can take some time. Therefore, it is a good practice to run first your algorithm on a small sample of your training and validation data before to run it on the entire set of data.


##### Image representation

Your first task will be to built a representation of the data, i.e. a feature vector which values quantify the contents of the image. We will see latter that, using Deep Convolutional Neural Networks, we can learn an efficient representation directly using raw pixel intensities as inputs. Here, we will just represent the images by two alternative representations.
+ A first representation is built using the raw data by simply resizing an input image to a fixed size (here $32 \times 32$ pixels) and then by flattening the RBG pixel intensities into a single vectors of numbers.
+ A second representation is built from the color histogram that characterizes the color distribution of the image. For this representation a color conversion into the HSV color space could be useful.

**Complete the functions below to build such representations**

Some helping functions :
+ With Numpy:
    + [Array flattening](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.flatten.html)
    
+ With OpenCV:

     + [Geometric transformations on images](http://docs.opencv.org/trunk/da/d6e/tutorial_py_geometric_transformations.html) 
     + [Colorspace conversion](http://docs.opencv.org/trunk/df/d9d/tutorial_py_colorspaces.html) 
     + [Histogram in OpenCV](http://docs.opencv.org/trunk/de/db2/tutorial_py_table_of_contents_histograms.html)
    
+ With Scikit-image :
    + [Loading an image from a file](https://scikit-image.org/docs/stable/user_guide/data_types.html)
    + [Image transformations](https://scikit-image.org/docs/dev/user_guide/transforming_image_data.html)
    + [Colorspace conversion](https://scikit-image.org/docs/dev/user_guide/transforming_image_data.html#color-manipulation)
     
    

Le code ci-dessous charge les dépendances utiles pour la suite.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os

In [ ]:
# TO DO : complete other import modules
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into a list of raw pixel intensities
    # TO COMPLETE

    return res



def build_HSV_color_histogram_vector(image,bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using the supplied number of `bins` per channel and return it as a feature vector

    # TO COMPLETE
    
    return res

image_to_feature_vector(None)

#### Dataset prepatation and feature extraction
The dataset has to be prepared for feature extraction :
+ Three lists will be initialized to store the raw pixel representation, the color distribution (histogram) representation and the class labels themselves.
+ Then, the lists will be completed by extracted the features from the dataset (don't forgot to begin by testing your program on the sample dataset before to apply it on the whole dataset).

For this step, we can used the paths.py file [here](https://github.com/jrosebr1/imutils/blob/master/imutils/paths.py)




In [ ]:
from paths import *

print("Describing images...")
imagePaths = list(list_images('./data/DogCatChallenge/sample'))

# initialize the raw pixel intensities matrix, the features matrix and labels list
rawImages_features = []
histogram_features = []
labels = []

# TO COMPLETE

#### Dataset splitting into training and validation dataset

The last common step in machine learning will be to split the training dataset into training dataset and validation dataset. For this, you could use some functions from scikit-learn :

  + [Cross validation](http://scikit-learn.org/stable/modules/cross_validation.html)
    
    
 In our case, the split of the dataset into training, validation and test set has been done but you can try to build another sample dataset using the fonctions of scikit-learn. In particular, if you are note familiar to **cross validation**, take the time to read carefully the previous documentation.

In [ ]:
from sklearn.model_selection import train_test_split

# TO COMPLETE : building of another sample dataset

#### Classification using the K-Nearest Neighbor (k-NN) classifier


In order to build this simple image classification pipeline, you will use the k-Nearest Neighbor (k-NN) classifier which is a very simple machine learning/image classification approach, rarely used in practice but its simplicity will enable you to get quickly an idea of the image classification pipeline.

The principle of the [k-NN classifier](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) can be summarize by the following principle : “Tell me who your neighbors are, and I’ll tell you who you are”. Given an unknown image and a training dataset, the k-NN classifier will compare this image to every single image of the training dataset and will predict the label by finding the most common class among the k-closest examples.

Thus to apply the k-nearest Neighbor classification, we need to define a distance metric or a similarity function. For the sake of simplicity, here we will consider the euclidian distance but other distances can be used according to the targeted type of data.

$d(\mathbf{p},\mathbf{q})=\sqrt{\sum_{i=1}^{N}(q_i-p_i)^2}$

with $\mathbf{p}$ and $\mathbf{q}$ two data vectorial representations.

You can now train and predict a k-Nearest Neighbor classifier on your different splits. Once again, we suggest you to have a look on the scikit-learn documentation :
+ [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)
+ [Example](http://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py)
+ [An introduction to machine learning with scikit-learn](http://scikit-learn.org/stable/tutorial/basic/tutorial.html#learning-and-predicting)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# training and evaluation of a k-NN classifer on the raw pixel representation

# TO COMPLETE

print("Raw pixel representation accuracy:")

# training and evaluation of  a k-NN classifer on the color histogram representation

# TO COMPLETE

print("Color histogram representation accuracy:")


**Take the time to compare and to discuss with the others your obtained score !**

#### Validation sets for Hyperparameter tuning

The k-nearest neighbor classifier requires a setting for k. What value have you chosen?  How ? Another parameter of this classifier is the choice of the distance functions we could have used. These choices are called **hyperparameters** and their setting is one of the main issue of the design of many Machine Learning algorithms.

To choose these hyperparameter values, the principle is to try out many different values and to see what works best. The main question is on which data ? On the test set ? On the training set ? In fact, the idea is to split your training set in two: a slightly smaller training set, and what we call a **validation set** as explained in the course.

The scikit-learn library has two methods that can perform hyperparameter search and optimization: **Grid search** and **Randomized Search**. Let's have a look on their documentations :
 + [Tuning the hyper-parameters of an estimator](http://scikit-learn.org/stable/modules/grid_search.html)
 
 
You will now apply these two methods to tune the hyperparameters of our k-NN classifier :
 1. First, define a dictionary of parameters which contains two keys :
     + `n_neighbors` the number of nearest neighbors $k$ in the k-NN algorithm that we will vary in the range $[1, 29]$ (if your sample dataset is sufficient)
     + `metric` the distance function/similarity metric for k-NN that can be either the Euclidean distance either the Manhattan/City block distance.
 2. Then, apply Grid Search and Randomized Search



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import time

print("Building training/testing split...")

# TO COMPLETE

params = {"n_neighbors": np.arange(1, 29, 1),"metric": ["euclidean", "cityblock"]}

# tune the hyperparameters via a cross-validated grid search of the KNN classifier on raw features

print("Tuning hyperparameters via grid search on raw features")
# TO COMPLETE
start = time.time()
# TO COMPLETE

# evaluate the best grid searched model on the testing data 

# TO COMPLETE : print the time and the obtained best parameters

# tune the hyperparameters via a cross-validated randomized search of the KNN classifier on raw features

print("Tuning hyperparameters via randomized search on raw features")
# TO COMPLETE
start = time.time()
# TO COMPLETE

# evaluate the best grid searched model on the testing data 

# TO COMPLETE : print the time and the obtained best parameters

# tune the hyperparameters via a cross-validated grid search of the KNN classifier on histo features

print("Tuning hyperparameters via grid search on histo features")
# TO COMPLETE
start = time.time()
# TO COMPLETE

# evaluate the best grid searched model on the testing data 

# TO COMPLETE : print the time and the obtained best parameters

# tune the hyperparameters via a cross-validated randomized search of the KNN classifier on raw features

print("Tuning hyperparameters via randomized search on histo features")
# TO COMPLETE
start = time.time()
# TO COMPLETE

# evaluate the best grid searched model on the testing data 

# TO COMPLETE : print the time and the obtained best parameters



#### Cross-validation

Sometimes, the size of your training data (and therefore also the validation data) is small and, in this case, the technique used for hyperparameter tuning is **cross-validation**. The main principle is to iterate over different validation sets and to average the performance across these. For example, in a 5-fold cross-validation, we split the training data into 5 equal folds, use 4 of them for training, and 1 for validation. We would then iterate over which fold is the validation fold, evaluate the performance, and finally average the performance across the different folds.

The scikit-learn library has also methods for this kind of cross validation :
+ [Cross-validation: evaluating estimator performance](http://scikit-learn.org/stable/modules/cross_validation.html#)

Here, you will just apply  **m-fold cross validation**. In particular, you have to write a 10-fold cross-validation run for the parameter k of our k-NN algorithm on the training dataset. You will just consider old value of $k$ on the range $[1,29]$


In [ ]:
from sklearn.model_selection import cross_val_score


# creating odd list of K for KNN
params = {"n_neighbors": np.arange(1, 29, 2)}

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation on the KNN classifier with raw features
print(" 10-fold cross validation on the KNN classifier with raw features")

# TO COMPLETE

print("Plotting the misclassification error versus k")
    
# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k =0
# TO MODIFY
print "The optimal number of neighbors is %d" % optimal_k

# plot misclassification error vs k
plt.plot(params["n_neighbors"], MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()



# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation on the KNN classifier with raw features
print(" 10-fold cross validation on the KNN classifier with hist features")

# TO COMPLETE

print("Plotting the misclassification error versus k")
    
# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k =0
# TO MODIFY
print "The optimal number of neighbors is %d" % optimal_k

# plot misclassification error vs k
plt.plot(params["n_neighbors"], MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()







#### Summary of Part 1

    
+ <span style="background-color:lightgreen">Introduction of the problem of **Image Classification** </span>.
+ <span style="background-color:lightgreen">Introduction and building of a **first simple classifier: the k-NN classifier**.</span> As seen, this classifier is very simple to implement and understand and one of its advantage is that is take no time to train since the training step consists in the storing and the indexing of the training data. Nevertheless, the test time should be expensive since it consists in comparing the target data to every training data. **This behavior is contrary to what is expected** since, in practice, we are often more careful with the test time efficiency that to the training time one.
+ <span style="background-color:lightgreen">Introduction of the **supervised machine learning basic pipeline** : preparation of the data and splitting into **training, validation and test sets**, **training of a model** on a labelled training dataset, **hyperparameters tuning** on the validation set, **evaluation of the model accuracy** on the test set.</span>
+ <span style="background-color:lightgreen">Introduction to the **cross-validation** procedure.</span>


<span style="background-color:lightblue"> You have finished the part 1 ! Complete the dashboard on the global MsTeams team.</span>

## Part 2 : Using Hand-Crafted Features

In this part, you will also follow the traditional image recognition approach but you will try to improve your image representation by using better features than the raw representation and the color distribution. Indeed, raw pixel data is hard to use for machine learning and for comparing images in general due to the different challenges explained before.
As a consequence, the computer vision community had studied and proposed a wide range of robust and discriminative features such as HOG (Histogram of Oriented Gradients), SIFT (Scale-Invariant Feature Transform) or SURF(Speeded Up Robust Features) among others. These features are often refered as **hand-crafted features**. Have a look on the slides on a good tutorial on features for image classification at ECCV 2010 : [http://ufldl.stanford.edu/eccv10-tutorial/](http://ufldl.stanford.edu/eccv10-tutorial/).

In this part, we will use the discriminative bag of visual words (BoVW) approach to represent the content of the target images.  Bag of visual words (BoVW) is a popular technique for image classification inspired by models used in natural language processing and texture recognition. BoVW downplays word arrangement (spatial information in the image) and classifies based on an histogram of the frequency of visual words in image content. The set of visual words forms a visual vocabulary, which is constructed by clustering a large corpus of features. A first step will thus consist in building a visual vocabulary by clustering a large set of local features (you will used DENSE SIFT) extracted from our training image corpus.

![Visualvocabularypipeline](images/bagoffeatures_visualwordsoverview.png)

Source : [mathwoks](https://fr.mathworks.com/help/vision/ug/image-classification-with-bag-of-visual-words.html?requestedDomain=www.mathworks.com)

Then, the typical BoVW pipeline for representing an image consists in :
1. extracting the local features from the image,
2. encoding the local features to the corresponding visual words
3. performing spatial pooling.

Then, all the images of your training set are described with the BoVW representation and on the same way than in the previous part you can train any off-shelf classifier to classify images. Here you will train a SVM.

![BovWrepresentation](images/yfA1C.png)

For this part we suggest to use the OpenCV and scikit-learn libraries.

Some documentation :
 + [Sift and OpenCV](https://docs.opencv.org/master/da/df5/tutorial_py_sift_intro.html)
 + A small tutorial on Bag of visual words in python [here](https://medium.com/@aybukeyalcinerr/bag-of-visual-words-bovw-db9500331b2f)


### Visual Vocabulary Building

In [ ]:
import cv2
import imutils 
import numpy as np
import os


# Here you have to extract SIFT features on all images of the training set and to make a k-means clustering to build the visual vocabulary.


# Get the path of the training set
# TO COMPLETE 


# Get all the path to the images and save them in a list
# image_paths and the corresponding label in image_classes
image_paths = []
image_classes = []
# TO COMPLETE

# Create feature extraction and keypoint detector objects using opencv 

# TO COMPLETE

# List where all the descriptors are stored
des_list = []

# TO COMPLETE 
    
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][1]
for image_path, descriptor in des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))  

# Perform k-means clustering
k = 100
# TO COMPLETE 


At the end of this first step, you have build your vocabulary of visual words. You will now use the vocabulary in order to build a representation of each image as a bag of visual words, i.e. each image will be described as an histogram of visual words.

### BoVW pipeline 
Here you have to write the BoVW pipeline and to apply it to each image of the training dataset. 

In [ ]:
# TO COMPLETE



### Classification using the BoVW representation and linear SVMs

Now, your objective is to train and test a dog and cat classifier using the BoVW representation and SVMs. The good practices of the Part 0 have to be used. 
Some references :

 + [http://scikit-learn.org/stable/modules/svm.html](http://scikit-learn.org/stable/modules/svm.html)
 + [http://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html](http://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html)
 + [http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py)

In [ ]:
from sklearn import svm


#### Summary of Part 2

    
+ <span style="background-color:lightgreen">Introduction of classical **hand crafted feature representation** used in computer vision </span>.
+ <span style="background-color:lightgreen"> Knowledge on OpenCV.</span> 
+ <span style="background-color:lightgreen"> Building of another **supervised machine learning basic pipeline** </span>
+ <span style="background-color:lightgreen"> A important family of classifiers **SVMs**</span>

<span style="background-color:lightblue"> You have finished the part 2 ! Complete the dashboard on the global MsTeams team.</span>

## Part 3 : Using Convolutional Neural Networks

In this part, you are going to use convolutional neural networks (CNNs) and deep learning in order to build your image classifier. You will use the [Keras framework](https://keras.io/) which is a high-level neural networks API, written in Python and capable of running on top of [TensorFlow](https://www.tensorflow.org/), [CNTK](https://github.com/Microsoft/CNTK), or [Theano](http://www.deeplearning.net/software/theano/). It was developed with a focus on enabling fast experimentation and as a consequence it is a good choice for this course. Various other frameworks are available and can also be used such as [Caffe framework](http://caffe.berkeleyvision.org/), [Torch](http://torch.ch/) or [DeepLearning4j](https://deeplearning4j.org/) among others. Another important deep learning framework is [pytorch](https://pytorch.org/).


### Keras with sample data from the Dogs and cats recognition challenge

In this part, you will have to use Keras in order to
 + Build and train a small network from scratch
 + Use the bottleneck features of a pre-trained network
 + Fine-tune the top layers of a pre-trained network

The work will be done on a **sample dataset** (sampleDeep) of the initial Kaggle challenge that contains a training set composed of 1000 images of cats and 1000 images of dogs and a validation set, used to evaluate our models, that contains 400 additional samples from each class.

At the end, you could apply this approach on the whole dataset but it will imply to have other computing ressources than just your own computer.



#### Data preparation and loading

As for the previous classifiers, data preparation is also required when working with convolutional neural networks and deep learning models. You will use the [*ImageDataGenerator class*](https://keras.io/preprocessing/image/) that defines the configuration for image data preparation but also for data augmentation, a step often necessary for deep learning. In the code below, you will have to create and configure an `ImageDataGenerator` and to fit it on your data. In this example, we will use the sample dataset of the Dogs and Cats challenge. We consider that you have a training directory and a validation directory setup in this manner :

    train_dir/
        dog/
        cat/
    val_dir/
        dog/
        cat
 This is the case of the `sampleDeep` dataset.
 

In [ ]:
# without augmentation, only rescaling

from keras.preprocessing.image import ImageDataGenerator

# definition of the number of samples propagated through the network at each step
batch_size = 16

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/DogCatChallenge/sampleDeep/train'
validation_data_dir = 'data/DogCatChallenge/sampleDeep/valid'

# create and configure an ImageDataGenerator for the training data with only rescaling to 0..1
train_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,  # the target directory
        target_size=(img_width, img_height),  # all images will be resized to 150x150
        batch_size=batch_size) 

# create and configure an ImageDataGenerator for the validation data with only rescaling to 0..1
test_datagen =  ImageDataGenerator(rescale=1./255)

valid_generator = test_datagen.flow_from_directory(
        validation_data_dir,  # the target directory
        target_size=(img_width, img_height),  # all images will be resized to 150x150
        batch_size=batch_size) 



Only few training examples are available in the `sampleDeep` dataset. In order to make the most of these training examples, a current approach is to **augment** them via a number of random transformations, so that our model would never see twice the exact same picture. This augmentation step helps prevent overfitting and helps the model generalize better.


In [ ]:
# with augmentation 

from keras.preprocessing.image import ImageDataGenerator

# definition of the number of samples propagated through the network at each step
batch_size = 16

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/DogCatChallenge/sampleDeep/train'
validation_data_dir = 'data/DogCatChallenge/sampleDeep/valid'

# Create and configure an ImageDataGenerator for the training data
# TO DO :  augmentation of the training data using rotation, horizontal and vertical shift, shearing tranformation, zooming 

# TO COMPLETE

# TO DO : Create and configure an ImageDataGenerator for the validation data
# Only rescaling for validation data

# TO COMPLETE

# TO DO : generator that will read pictures found in the train dataset directory and that will indefinitely generate
# batches of augmented image data

# TO COMPLETE

# TO DO : Similar generator for validation data

# TO COMPLETE


## Build a model from scratch

Models can be build easily with the Keras API. Here we will use the Sequential model API :
+ [https://keras.io/getting-started/sequential-model-guide/](https://keras.io/getting-started/sequential-model-guide/)


Here, you will build a convolutional neural network which is ,by design, one of the best models available for most "perceptual" problems (such as image classification), even with very little data to learn from.

In the code below, you have to build a model composed of 3 convolution layers with a ReLU activation and followed by max-pooling layers.
In order to write the code, have a look on the [documentation on the different kinds of layers available in Keras](https://keras.io/layers/about-keras-layers/)




In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()

# First convolutional layer

# TO COMPLETE

# Second  convolutional layer

# TO COMPLETE
 
# Third convolutional layer    

# TO COMPLETE

# Adding of two fully-connected layers 

# TO COMPLETE

# single unit and sigmoid activation, which is perfect for a binary classification. 

model.add(Dense(1))
model.add(Activation('sigmoid'))

# Use of the binary_crossentropy loss to train our model, of the rmsprop optimizer and the accuracy metrics

# TO COMPLETE



### Visualizing a model

It may be sometimes useful to visualize in a schematic way a model architecture. You can do it with different approaches on Keras :

+ using the `plot_model` built-in function : see [this small tutorial](https://www.machinecurve.com/index.php/2019/10/07/how-to-visualize-a-model-with-keras/) or the [official documentation](https://keras.io/api/utils/model_plotting_utils/).
+ using [TensorBoard](https://www.tensorflow.org/tensorboard) if you are using the tensorflow backend on Keras. You can also find some documentations on this small [tutorial](https://www.machinecurve.com/index.php/2019/12/03/visualize-keras-models-overview-of-visualization-methods-tools/#visualizing-model-architecture-tensorboard)

In [ ]:
# TO DO
# Try the previous tools to display your model architecture.

### Training a model

We can now use some defined generators to train your build model.

In [ ]:
# augmentation configuration use for training:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# augmentation configuration use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

# fit the generator to your data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# model training
# TO COMPLETE

# saving the learned model
model.save_weights('first_try.h5')


#### Carefully have a look on the results and on the diffferent metrics and their obtained values. What is your interpretation of the results ?

You can now apply this model to any new image. For instance, in the code below, you have to apply the model on different images of the test dataset. 

In [ ]:
%matplotlib inline

from skimage import data, io
from matplotlib.pyplot import imshow
from keras.preprocessing import image
import numpy as np

img_path = 'data/DogCatChallenge/test1/10002.jpg'

ima=io.imread(img_path)
imshow(ima)

# image loading and transformation to keras

# TO COMPLTE

# prediction on the image
print("Prediction")

# TO COMPLETE


## Use a pretrained Convnet model

In practice, very few people train an entire Convolutional Network from scratch (with random initialization), because it is relatively rare to have a dataset of sufficient size. Instead, in image classification, it is common to use networks pre-trained on a large dataset (such as ImageNet)  and to use it either as an initialization of as a fixed feature extractor for the task of interest (**transfer learning**). Indeed, these networks have already learned features that are useful for most computer vision problems, and leveraging such features would allow us to reach a better accuracy than any method that would only rely on the available data.

Different strategies can be used in transfer learning scenarios :

1. The ConvNet, trained on a large image dataset such as Imagenet, is used as a fixed feature extractor. In this case, the pipeline consists in taking the pre-trained ConvNet, removing the last fully connected layer and that by treating the rest of the ConvNet architecture as a fixed feature extractor for the new dataset
2. Fine Tuning of the ConvNet. In this case,  the weights of a part of the pretrained network are fine-tuned by continuing the backpropagation. As it as been observed that the first features of a ConvNet contain more generic features (e.g. edge detectors or color blob detectors) that should be useful to many tasks and that later layers become progressively more specific to the details of the classes contained in the original dataset, only a higher portion of the network is fine-tuned.


### ConvNet as a fixed feature extractor   

In our case, the ImageNet dataset contains several "cat" classes (persian cat, siamese cat...) and many "dog" classes among its total of 1000 classes. As a consequence any model pre-trained on ImageNet will already have learned features that are relevant to our classification problem. 

In particular, we will use the VGG16 architecture which won the 2014 Imagenet competition, and is a very simple model to create and understand. The VGG Imagenet team created both a larger, slower, slightly more accurate model (VGG 19) and a smaller, faster model (VGG 16). We will be using VGG 16 since the much slower performance of VGG19 is generally not worth the very minor improvement in accuracy.

![VGG16](images/vgg16.png)

Source : [https://blog.heuritech.com/2016/02/29/a-brief-report-of-the-heuritech-deep-learning-meetup-5/](https://blog.heuritech.com/2016/02/29/a-brief-report-of-the-heuritech-deep-learning-meetup-5/)

In the code below, the strategy will consist in instantiating only the convolutional part of the model (using the *include_top* argument) (see the [Keras documentation on VGG16](https://keras.io/applications/#vgg16)) and in running this model on our own training and validation data once by recording the output in two numpy arrays. Then, you will train a small fully-connected model on top of the stored features.

Some references :
 + VGG models : [http://www.robots.ox.ac.uk/~vgg/research/very_deep/](http://www.robots.ox.ac.uk/~vgg/research/very_deep/)
 



In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications


# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/DogCatChallenge/sampleDeep/train'
validation_data_dir = 'data/DogCatChallenge/sampleDeep/valid'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


# Function that instanciates the convolutional part of the VGG16 pre-trained model on Imagenet and that runs it on our training and validation data

def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build and load the VGG16 network without the fully connected layers
    # TO COMPLETE

    # preparation of the training data
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    # Generation of the predictions for the input samples from the training data generator and return them as a numpy array that we can saved
    bottleneck_features_train = # TO COMPLETE
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    # preparation of the validation data
    
    # TO COMPLETE
    
    # Generation of the predictions for the input samples from the validation data generator and return them as a numpy array that we can saved
    bottleneck_features_validation = # TO COMPLETE
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

    
# Function that trains a small fully-connected model on top of the stored previous features
    
    
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    # Building of the small fully-connected model
    
    # TO COMPLETE
    
    # Configuration of the learning process
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    # Training of the model
    # TO COMPLETE
    
save_bottlebeck_features()
train_top_model()

#### Carefully have a look on the results and on the diffferent metrics and their obtained values. What is your interpretation of the results ?
Try to apply this model of some unknown images, even images without dogs ar cats.

## Fine-tuning the top layers of a a pre-trained network

We will now try to "fine-tune" the last convolutional block of the VGG16 model. It consist in starting from a trained network (the VGG16 network), then re-training it on a new dataset using very small weight updates. In our case, this can be done in 3 steps:
+ Instantiate the convolutional base of VGG16 and load its weights.
+ Add our previously defined fully-connected model on top, and load its weights.
+ Freeze the layers of the VGG16 model up to the last convolutional block


In [ ]:
from keras import applications
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import keras

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150
keras.backend.set_image_dim_ordering('tf')

train_data_dir = 'data/DogCatChallenge/sampleDeep/train'
validation_data_dir = 'data/DogCatChallenge/sampleDeep/valid'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# creation of the base VGG pre-trained model
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
# TO COMPLETE

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)


# creation of a real model from vgg
new_model = Sequential()
for l in model.layers:
    new_model.add(l)


# concatenation of the base model with the top model
# TO COMPLETE


# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)

# TO COMPLETE

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model

# TO COMPLETE


#### Carefully have a look on the results and on the diffferent metrics and their obtained values. What is your interpretation of the results ?
Try to apply this model of some unknown images, even images without dogs ar cats.

#### Summary of Part 3

    
+ <span style="background-color:lightgreen">Introduction to representation learning </span>.
+ <span style="background-color:lightgreen"> Introduction to deep learning and CNN.</span> 
+ <span style="background-color:lightgreen"> Practice on Keras </span>
+ <span style="background-color:lightgreen"> Introduction to transfer learning and fine tuning </span>

<span style="background-color:lightblue"> You have finished the part 2 ! Complete the dashboard on the global MsTeams team.</span>

## Opening the black box [Optional]

Deep neural network models are often considered as  black-boxes and their performances come at the price of loss of interpretability. Indeed, they fail to provide explanations on their predictions. In high-risk domains, e.g., health care, or in a context of production, it is crucial to build trust in a model and being able to understand its behavior. This sub-fied of artificial intelligence is known as XAI (eXplainable Artificial Intelligence) (see for instance this [tutorial](https://sites.google.com/view/www20-explainable-ai-tutorial) or the DARPA initiative [here](https://www.darpa.mil/attachments/XAIProgramUpdate.pdf)).

In particular different approaches are now well established as tools to explain a model or a model decision. You can test them on your different classification models.



#### Activation Maximization

See this [tutorial](https://www.machinecurve.com/index.php/2019/11/18/visualizing-keras-model-inputs-with-activation-maximization/) to test it on your models.


In [ ]:
# TO DO
# Activation maximization on your models.




#### Saliency maps

See this [tutorial](https://www.machinecurve.com/index.php/2019/11/25/visualizing-keras-cnn-attention-saliency-maps/) to test it.

In [ ]:
# TO DO
# Saliency maps on your models.

If you are interested to go deeper, a very nice tutorial is available [here](https://interpretablevision.github.io/)

# Part 4 : Your turn on a more challenging case

Do the same job but this time on the PET Dataset : [http://www.robots.ox.ac.uk/~vgg/data/pets/](http://www.robots.ox.ac.uk/~vgg/data/pets/) which contains 37 category of pets with roughly 200 images for each class.

# Sources and references

+ This study case is inpired from the Lesson 1 of the fast.ai's online course, Practical Deep Learning For Coders : [http://course.fast.ai/](http://course.fast.ai/)
+ Others sources :
    + Stanford CS231n course on Convolutional Neural Networks for Visual Recognition : [http://cs231n.stanford.edu/](http://cs231n.stanford.edu/)
    + Keras blog post on building image classification models [here](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)